<a href="https://colab.research.google.com/github/Camille-saran/puns_analogies/blob/main/dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SPARQLWrapper
!pip install wikipedia
!pip install wptools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 7.8 MB/s 
     |████████████████████████████████| 41 kB 488 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=d9510bd33c297ae07db8f1df95276a9a02636012b9fe1454ec6d45b09fbeb634
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 233 kB 9.2 MB/s 
  Created wheel for pycurl: filename=pycurl-7.45.1-cp37-cp37m-linux_x86_64.whl size=300983 sha256=d3fbb16080601cc62e8a3ea8914c996507b2949c3b92f7b84107916d653a4a74
  Stored in directory: /root/.cache/pip/wheels/b4/fe/1f/e6cae4153dbc082271bd57

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import wikipedia
import wptools
import re

In [ ]:
#Here, write for k the number of articles you want to be retrieved
k = 60

In [ ]:
def get_articles(query):
    """This function return a list of articles from a given category query
    Input: sparql query
    Output: list of article from the given category in a dictionary
    """
    #create SPARQLWrapper, with an agent parameter to avoid 403 HTTP error
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36')
    #query on the database
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    #return the list in list_article
    list_article = sparql.query().convert()
    
    return list_article

In [ ]:
def get_list_id(list_article):
    """From an input list of article, this function return the corresponding id from article"""
    list_article_id = []
    #from the dictionary is retrieved the uris of each article
    for article in list_article["results"]["bindings"]:
        list_article_id.append(article["item"]["value"])
    #only returns the k articles 
    return list_article_id[:k+140]

In [ ]:
def get_data(query):
    
    list_article = get_articles(query)
    list_article_id =  get_list_id(list_article)
    
    list_idi = []
    list_title = []   
    #parcourt la liste des articles
    for x in list_article_id:
        #créé un objet m qui va à chaque tour de boucle recherche la phrase ci dessous
        m = re.match(r"http://www.wikidata.org/entity/([\w]+)",x)
        #récupère l'id de chaque article Q+numéro
        idi = m.group(1)
        #ajoute l'id à la liste de base
        list_idi.append(idi)
        page = wptools.page(wikibase= idi)
        try:
            page.get_wikidata()
            
        except wikipedia.exceptions.PageError:
            continue
            
        except wikipedia.exceptions.DisambiguationError:
            continue
            
        except LookupError:
            continue
        try : 
            list_title.append(page.data['title'].replace("_", " "))
        except KeyError:
            list_title.append("NaN")
        
    return list_title

In [ ]:
query_objet = "SELECT DISTINCT ?item ?itemLabel WHERE {SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }{SELECT DISTINCT ?item WHERE {?item p:P279 ?statement0.?statement0 (ps:P279/(wdt:P279*)) wd:Q14745.}LIMIT 300}}"
query_perso = "SELECT DISTINCT ?item ?itemLabel WHERE {SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". } {SELECT DISTINCT ?item WHERE {?item p:P31 ?statement0.?statement0 ps:P31 wd:Q2393196.}LIMIT 1000}}"
query_animal = "SELECT DISTINCT ?item ?itemLabel WHERE {SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }{SELECT DISTINCT ?item WHERE {?item p:P279 ?statement0.?statement0 (ps:P279/(wdt:P279*)) wd:Q729. }LIMIT 1000}}"
query_celebrity = "SELECT DISTINCT ?item ?itemLabel ?fame WHERE {SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE]\". }{SELECT DISTINCT ?item WHERE {?item wikibase:sitelinks ?fame FILTER(?fame > 100) .{?item p:P106 ?statement0.?statement0 (ps:P106/(wdt:P279*)) wd:Q33999.}UNION{?item p:P106 ?statement1.?statement1 (ps:P106/(wdt:P279*)) wd:Q177220.}UNION{?item p:P106 ?statement2.?statement2 (ps:P106/(wdt:P279*)) wd:Q1028181.}UNION{ ?item p:P106 ?statement3. ?statement3 (ps:P106/(wdt:P279*)) wd:Q82955.}UNION{?item p:P106 ?statement4.?statement4 (ps:P106/(wdt:P279*)) wd:Q205375.}}LIMIT 100}}"
query_insults = "SELECT DISTINCT ?item ?itemLabel WHERE {SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }{SELECT DISTINCT ?item WHERE {?item p:P31 ?statement0.?statement0 (ps:P31/(wdt:P279*)) wd:Q571571.}LIMIT 400}}"
query_travail = "SELECT DISTINCT ?item ?itemLabel WHERE {SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }{SELECT DISTINCT ?item WHERE {?item p:P31 ?statement0.?statement0 (ps:P31/(wdt:P279*)) wd:Q6958747.}}}"

In [ ]:
list_O = get_data(query_objet)
list_P = get_data(query_perso)
list_A = get_data(query_animal)
list_C = get_data(query_celebrity)
list_I = get_data(query_insults)
list_T = get_data(query_travail)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  wikidata: <dict(3)> instance of (P31), facet of (P1269), Google ...
  wikidata_pageid: 96344317
  wikidata_url: https://www.wikidata.org/wiki/Q98058427
}
www.wikidata.org (wikidata) Q63522577
www.wikidata.org (labels) P31|Q571571|P2671
fuck you (en) data
{
  aliases: <list(3)> fuck yourself, go fuck yourself, transitive fuck
  claims: <dict(2)> P31, P2671
  description: popular vulgar expression, a form of the profanity ...
  label: fuck you
  labels: <dict(3)> P31, Q571571, P2671
  modified: <dict(1)> wikidata
  requests: <list(2)> wikidata, labels
  title: fuck_you
  what: insult
  wikibase: Q63522577
  wikidata: <dict(2)> instance of (P31), Google Knowledge Graph ID...
  wikidata_pageid: 63248059
  wikidata_url: https://www.wikidata.org/wiki/Q63522577
}
www.wikidata.org (wikidata) Q1527157
www.wikidata.org (labels) P921|Q1371427|Q22947|Q16607692|P31
None (en) data
{
  claims: <dict(2)> P31, P921
  labels

In [ ]:
list_O = [x.lower() for x in list_O if x!="NaN"]
list_P = [x.lower() for x in list_P if x!="NaN"]
list_A = [x.lower() for x in list_A if x!="NaN"]
list_C = [x.lower() for x in list_C if x!="NaN"]
list_I = [x.lower() for x in list_I if x!="NaN"]
list_T = [x.lower() for x in list_T if x!="NaN"]

In [ ]:
list_A

['politically exposed person',
 'passenger',
 'arahitogami',
 'quirites',
 'jōrurikatari',
 'municipality president (switzerland)',
 '9x generation',
 'migrant',
 'midget',
 'huaxia',
 'unperson',
 'black disciples',
 'asian people',
 'healthy adult',
 'cancer survivor',
 'strangers (parliament of the united kingdom)',
 'marabout',
 'cro-magnon man',
 'alien (law)',
 'neurotypical',
 'californios',
 'neighbor',
 'lyubomudry',
 'anonymous',
 'freeman on the land',
 'digital addict',
 'adept',
 'oldest people',
 'unidentified decedent',
 'androgynos',
 'red sticks',
 'gentry',
 'citizen',
 'safety officer',
 'conscientious objector',
 'other person',
 'list of guantanamo bay detainees',
 'night owl',
 'vulnerable population',
 'descendants of pushkin',
 'busybody',
 'groundling',
 'know-it-all',
 'mulatto',
 'white people',
 'colleague',
 'gittite',
 'pre-adamite',
 'talmid chakham',
 'centralien',
 'doghunters',
 'mahasiddha',
 'tumtum (judaism)',
 'mormon spectrums of orthodoxy and pra

Ce notebook permet d'obtenir un dataframe avec pour chaque ligne : 
- un mot, 
- ses synonymes, 
- ses antonymes, 
- ses hyperonymes/hyponymes, s'il y en a

In [ ]:
#imoporter les librairies
import nltk
import pandas as pd
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#creation du dataframe vide
df = pd.DataFrame(columns=['word','lexical_field','syn','ant','hyper','hypo'])
list_ = []
list_ = list_O + list_A + list_C + list_I + list_P + list_T
df['word']= list_
for x, mot in enumerate(list_):
  if mot in list_A : 
    df.loc[x].at['lexical_field']='Animal'
  if mot in list_O : 
    df.loc[x].at['lexical_field']='Object'
  if mot in list_C : 
    df.loc[x].at['lexical_field']='Celebrite'
  if mot in list_I : 
    df.loc[x].at['lexical_field']='Insultes'
  if mot in list_P : 
    df.loc[x].at['lexical_field']='Anmal'
  if mot in list_T : 
    df.loc[x].at['lexical_field']='Travail'

df

,word,lexical_field,syn,ant,hyper,hypo
0,nightstand,Object,NaN,NaN,NaN,NaN
1,drawing board,Object,NaN,NaN,NaN,NaN
2,teapoy,Object,NaN,NaN,NaN,NaN
3,reading table,Object,NaN,NaN,NaN,NaN
4,salad bar,Object,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
854,jewelry designer,Travail,NaN,NaN,NaN,NaN
855,wine measurer,Travail,NaN,NaN,NaN,NaN
856,quantitative analyst,Travail,NaN,NaN,NaN,NaN
857,ecclesiastical guard,Travail,NaN,NaN,NaN,NaN


In [ ]:
for x, y in enumerate(list_):
    for synset in wn.synsets(y):
        ant_ = []
        syn_= []
        for l in synset.lemmas():
            syn_.append(l.name())
            if l.antonyms():
                ant_.append(l.antonyms()[0].name())
    try:
      My_sysn = wn.synsets(y)[0] 
      hyper_ = [str(x.lemma_names()) for x in My_sysn.hypernyms()]
      hypo_ = [str(x.lemma_names()) for x in My_sysn.hyponyms()]
    except:
      continue
    syn_ = [item for item in syn_ if item != y]
    try:
      df.iloc[x]['hypo']=hypo_
      df.iloc[x]['hyper']=hyper_
      df.iloc[x]['syn']=syn_
      df.iloc[x]['ant']=ant_
    except IndexError:
      continue
#liste_mot =  list(set(liste_mot))

In [ ]:
df.head(5)#df final

,word,lexical_field,syn,ant,hyper,hypo
0,nightstand,Object,NaN,NaN,NaN,NaN
1,drawing board,Object,NaN,NaN,NaN,NaN
2,teapoy,Object,NaN,NaN,NaN,NaN
3,reading table,Object,NaN,NaN,NaN,NaN
4,salad bar,Object,NaN,NaN,NaN,NaN


In [ ]:
df2 = df.dropna()

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/queries.csv'
with open(path, 'w') as f:
  df2.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
